In [5]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

ModuleNotFoundError: No module named 'utils'

To use Langchain with OpenAI models, you need to set up your OpenAI API key. You can add it to Colab's secrets manager under the "🔑" in the left panel. Give it the name `OPENAI_API_KEY`. Then you can access it in your notebook like this:

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_1')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo' # You can change this to other GPT versions

Now you can load a GPT model from Langchain. Here's an example using `ChatOpenAI`:

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.environ["OPENAI_MODEL_NAME"])

# You can now use the 'llm' object to interact with the GPT model
# For example:
# response = llm.invoke("Hello, how are you?")
# print(response.content)

Creating Agents
Define your Agents, and provide them a role, goal and backstory.
It has been seen that LLMs perform better when they are role playing.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

Creating Tasks
Define your Tasks, and provide them a description, expected_output and agent.

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

Creating the Crew
Create your crew of Agents
Pass the tasks to be performed by those agents.
Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
verbose=2 allows you to see all the logs of the execution.

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the current landscape of Artificial Intelligence
- Importance of staying updated on the latest trends and developments in AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- Natural language processing
- AI ethics and bias
- Automation in various industries

2. Key Players in the AI Industry:
- Google
- Amazon
- IBM
- Microsoft
- Tesla

3. Notew

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become an integral part of our lives, revolutionizing various industries and shaping the way we interact with technology. Staying updated on the latest trends and developments in AI is crucial for professionals in the tech industry, business owners looking to implement AI solutions, and students studying AI or related fields. In this blog post, we will explore the current landscape of AI, highlighting the latest trends, key players in the industry, and noteworthy news that is shaping the future of Artificial Intelligence.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
One of the most significant trends in AI is the continuous advancements in machine learning algorithms. Companies are leveraging machine learning models to analyze vast amounts of data, enabling them to make data-driven decisions and improve business processes. From recommendation systems to predictive analytics, machine learning is at the forefront of AI innovation.

### Natural Language Processing
Natural Language Processing (NLP) is another area of AI that has seen rapid development in recent years. NLP technologies enable machines to understand, interpret, and generate human language, opening up new possibilities for communication, customer service, and content creation. Chatbots, virtual assistants, and language translation services are just a few examples of NLP applications that are transforming industries.

### AI Ethics and Bias
As AI technology becomes more prevalent, the importance of addressing ethical concerns and biases in AI algorithms has come to the forefront. Ensuring that AI systems are fair, transparent, and accountable is essential to building trust and avoiding unintended consequences. Ethical AI frameworks and guidelines are being developed to guide the responsible use of AI technology.

### Automation in Various Industries
Automation powered by AI is reshaping various industries, streamlining operations, and improving efficiency. From manufacturing and healthcare to finance and retail, businesses are deploying AI-powered automation solutions to optimize processes, reduce costs, and enhance customer experiences. The rise of robotic process automation (RPA) and autonomous systems is driving the next wave of industrial revolution.

## Key Players in the AI Industry

### Google
Google is a major player in the AI industry, leveraging its vast data resources and research capabilities to develop cutting-edge AI technologies. From self-driving cars to natural language understanding, Google's AI initiatives span a wide range of applications and services, shaping the future of AI.

### Amazon
Amazon's AI efforts are focused on enhancing customer experiences through personalized recommendations, efficient logistics, and voice-enabled devices like Alexa. The e-commerce giant is investing heavily in AI research and development to stay ahead of the competition and drive innovation across its platform.

### IBM
IBM is a leader in enterprise AI solutions, offering businesses a suite of AI tools and services to accelerate digital transformation. From Watson AI to cloud-based AI services, IBM is empowering organizations to harness the power of AI for better decision-making and competitive advantage.

### Microsoft
Microsoft's AI strategy revolves around empowering individuals and organizations with AI capabilities through its Azure AI platform and cognitive services. From AI-driven productivity tools to intelligent cloud solutions, Microsoft is driving AI innovation across its product ecosystem.

### Tesla
Tesla is at the forefront of AI innovation in the automotive industry, pioneering self-driving technology and AI-powered electric vehicles. With advancements in autonomous driving and AI-based vehicle features, Tesla is redefining the future of transportation and sustainability.

## Noteworthy News in Artificial Intelligence

### Recent Breakthroughs in AI Research
The field of AI is constantly evolving, with researchers making breakthroughs in areas like reinforcement learning, computer vision, and neural network architectures. From AlphaGo's victory over human champions to GPT-3's language generation capabilities, AI research is pushing the boundaries of what machines can achieve.

### Impact of AI on Job Market
The rise of AI and automation is reshaping the job market, creating new opportunities and challenges for workers across industries. While AI technologies can enhance productivity and create new roles, they also have the potential to automate tasks and displace jobs that are susceptible to automation. Upskilling and reskilling workers for the AI-driven economy is essential to ensure a smooth transition.

### Ethical Concerns Surrounding AI Technology
Ethical considerations are a critical aspect of AI development and deployment, as AI systems can have far-reaching implications on society and individuals. Issues like bias in AI algorithms, data privacy, and algorithmic accountability are key concerns that need to be addressed to ensure the responsible and ethical use of AI technology. Stakeholders across industries are working towards developing ethical frameworks and guidelines to guide the ethical development and deployment of AI systems.

## Conclusion

As Artificial Intelligence continues to advance and permeate various aspects of our lives, staying informed on the latest trends, key players, and noteworthy news in the AI industry is essential. By understanding the implications of AI technology and its impact on businesses and society, we can harness the power of AI for positive outcomes and address the challenges that come with its adoption. Whether you are a professional in the tech industry, a business owner looking to implement AI solutions, or a student studying AI, keeping up with the latest developments in AI will empower you to navigate the evolving landscape of Artificial Intelligence.

Don't miss out on the opportunity to stay informed on the latest trends in Artificial Intelligence. Subscribe to our newsletter, connect with industry experts, and explore AI consulting services to unlock the full potential of AI in your business. The future of Artificial Intelligence is here, and it's up to us to shape it responsibly and ethically.

In [14]:
topic = "Machine Learning"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Machine Learning.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Machine Learning Blog Article

Introduction:
- Brief overview of Machine Learning
- Importance of Machine Learning in various industries

Key Points:
1. Latest Trends in Machine Learning:
- Advancements in deep learning algorithms
- Integration of Machine Learning in Internet of Things (IoT)
- Rise of Explainable AI for transparency and accountability

2. Key Players in Machine Learning:
- Google's TensorFlow framework
- Amazon Web Services (AWS) Machine Learning services
- Microsoft Azure Machine Lea

In [15]:
Markdown(result)

# The Fascinating World of Machine Learning

In today's rapidly evolving technological landscape, Machine Learning has emerged as a groundbreaking field that continues to shape various industries. Machine Learning, a subset of artificial intelligence, allows systems to learn from data and make decisions or predictions without being explicitly programmed. Its applications range from healthcare to finance, revolutionizing how businesses operate and how we interact with technology.

## Latest Trends in Machine Learning

One of the most exciting trends in Machine Learning is the advancements in deep learning algorithms. These algorithms mimic the human brain's structure, enabling machines to process complex data and recognize patterns with remarkable accuracy. Additionally, the integration of Machine Learning in the Internet of Things (IoT) has unlocked new possibilities for smart devices and connected systems. Moreover, the rise of Explainable AI addresses the need for transparency and accountability in automated decision-making processes, ensuring that users understand why a particular decision was made by an AI system.

## Key Players in Machine Learning

Leading the way in the Machine Learning industry are key players such as Google's TensorFlow framework, renowned for its flexibility and scalability in developing machine learning models. Amazon Web Services (AWS) offers a comprehensive suite of Machine Learning services, empowering businesses to leverage data for actionable insights. Microsoft Azure Machine Learning provides a user-friendly platform for building, training, and deploying Machine Learning models, catering to a wide range of industries.

## Noteworthy News in Machine Learning

Recent breakthroughs in natural language processing have unlocked new possibilities for human-machine interactions, enabling more intuitive communication between users and AI systems. In the healthcare sector, Machine Learning is revolutionizing diagnostics and personalized treatment plans, enhancing patient care and outcomes. Ethical considerations in AI and Machine Learning development have also gained prominence, prompting discussions on bias, privacy, and accountability in algorithmic decision-making processes.

In conclusion, Machine Learning continues to drive innovation across industries, shaping the way we work, communicate, and live. By staying informed on the latest trends, key players, and ethical considerations in Machine Learning, professionals in the tech industry, data scientists, and students can harness the power of AI for positive impact. Let us embrace the opportunities that Machine Learning offers and explore the endless possibilities of this transformative technology.